In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keyring
import os 
import snowflake.connector as sf_connector # ( https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-connect)
from snowflake.connector.pandas_tools import write_pandas # (https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-api#write_pandas)
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.evaluation import load_evaluator
from collections import defaultdict

import numpy as np
from tqdm import tqdm
import time
import re
import json

from io import BytesIO
import fitz 
from shapely.geometry import box
from shapely.ops import unary_union
from PIL import Image, ImageDraw
import cv2
from openai import OpenAI


# Set max rows to display in pandas DataFrame 200
pd.set_option('display.max_rows', 200)

In [4]:
open_ai_api_key = keyring.get_password("openai api", "api_key")
client = OpenAI(api_key = open_ai_api_key)

account_identifier = keyring.get_password('NC_Snowflake_Trial_Account_Name', 'account_identifier')
user_name = "EMHALDEMO1"
password = keyring.get_password('NC_Snowflake_Trial_User_Password', user_name)
database = "WASHING_MACHINE_MANUALS"
schema = "PUBLIC"

print("Account Identifier: ", account_identifier)
print("User Name: ", user_name)
print("Database: ", database)
print("Schema: ", schema)

try:
    connection_parameters = {
        "account_identifier": account_identifier,
        "user": user_name,
        "password": password,
        "role": "ACCOUNTADMIN",
        "warehouse": "COMPUTE_WH",
        "database": database,
        "schema": schema
    }
except:
        connection_parameters = {
        "account_identifier": account_identifier,
        "user": user_name,
        "password": password,
        "role": "ACCOUNTADMIN",
        "warehouse": "COMPUTE_WH",
        "database": "SNOWFLAKE",
        "schema": "CORTEX"
    }


# Connect to Snowflake
conn = sf_connector.connect(
    user=connection_parameters['user'],
    password=connection_parameters['password'],
    account=connection_parameters['account_identifier'],
    warehouse=connection_parameters['warehouse'],
    database=connection_parameters['database'],
    schema=connection_parameters['schema'],
    role=connection_parameters['role']
)

cursor = conn.cursor()
cursor.execute(f" CREATE DATABASE IF NOT EXISTS {database}; ")
cursor.execute(f" CREATE SCHEMA IF NOT EXISTS {database}.{schema}; ")
cursor.execute(f" USE DATABASE {database}; ")
cursor.execute(f" USE SCHEMA {schema}; ")


Account Identifier:  EPTJRCA-HWB83214
User Name:  EMHALDEMO1
Database:  WASHING_MACHINE_MANUALS
Schema:  PUBLIC


In [15]:
# Lets see the table
cursor.execute("""
    SELECT * 
    FROM DOCUMENTS;
""")

documents_df = cursor.fetch_pandas_all()
documents_df

,DOCUMENT_ID,DOCUMENT_NAME,DOC_VERSION,FILE_PATH,FILE_SIZE,CREATED_AT
0,1,WAV28KH3GB.pdf,N/A,.\Washer_Manuals\WAV28KH3GB.pdf,5686613,2025-04-22 01:52:03.390000-07:00
1,2,WGA1420SIN.pdf,N/A,.\Washer_Manuals\WGA1420SIN.pdf,3247850,2025-04-22 01:52:03.390000-07:00
2,3,WGG254Z0GB.pdf,N/A,.\Washer_Manuals\WGG254Z0GB.pdf,3291555,2025-04-22 01:52:03.390000-07:00
3,101,WAV28KH3GB.pdf,N/A,.\Washer_Manuals\WAV28KH3GB.pdf,5686613,2025-04-25 04:44:31.675000-07:00
4,102,WGA1420SIN.pdf,N/A,.\Washer_Manuals\WGA1420SIN.pdf,3247850,2025-04-25 04:44:31.675000-07:00
5,103,WGG254Z0GB.pdf,N/A,.\Washer_Manuals\WGG254Z0GB.pdf,3291555,2025-04-25 04:44:31.675000-07:00


In [5]:
# Lets see the table
cursor.execute("""
    SELECT * 
    FROM CHUNKS_LARGE;
""")

large_chunks_df = cursor.fetch_pandas_all()
large_chunks_df.head()

,CHUNK_ID,DOCUMENT_ID,PAGE_START_NUMBER,PAGE_END_NUMBER,CHUNK_ORDER,CHUNK_TEXT,EMBEDDING,CREATED_AT
0,1,1,0,3,0,Register your b M o ge s y n c t B e h f o r w...,"[0.048339844, 0.0158844, -0.015388489, -0.0038...",2025-04-22 01:53:38.352000-07:00
1,2,1,3,7,1,t. ¡ Up to an altitude of max. 4000 m above se...,"[0.038604736, 0.08459473, -0.023513794, -0.014...",2025-04-22 01:53:38.352000-07:00
2,3,1,7,10,2,on or lean against the appliance door. ▶ Do no...,"[0.074279785, 0.068359375, -0.0036392212, 0.02...",2025-04-22 01:53:38.352000-07:00
3,4,1,10,15,3,"ese instructions, your creases energy and wate...","[0.008338928, 0.00027441978, 0.044067383, 0.00...",2025-04-22 01:53:38.352000-07:00
4,5,1,15,22,4,ht. Water outlet connection types 4.6 Aligning...,"[0.052764893, 0.074157715, -0.033294678, 0.011...",2025-04-22 01:53:38.352000-07:00


In [7]:

prompt = """ 
    I will provide a long string of text in which i want you to identify all of the error codes and return them in a JSON format.
    It is likely that it doesn't contain any error codes
    The JSON format should be as follows:
    {
        "Error Codes": [
            {
                "Code": "<error code>",
                "Description": "<description of the error code>"
            },
            ...
        ]
    }

    Here is the text:
    
"""

idx = 7

response = client.responses.create(
    model="gpt-4o",
    instructions= prompt,
    input=large_chunks_df.loc[idx,"CHUNK_TEXT"],
)

print("Chunk: \n", large_chunks_df.loc[idx,"CHUNK_TEXT"])
print("")
print(response.output_text)

Chunk: 
 ing that has ¡ Laundry has absorbed been worn for a just a few odours hours Normal Dirt or light stains are vis- ¡ T-shirts, shirts or blouses ible are sweat-soaked or have been worn a few times ¡ Towels or bedding that has been used for up to one week Heavy Dirt or stains are clearly Tea towels, baby items or visible work clothing 11.4 Care symbols on the care labels Washing care symbols Symbol Washing process Recommended pro- gramme ⁠ ⁠ Normal Cotton ⁠ Gentle Easy-Care ⁠ Particularly gentle Delicates/Silk for washing by hand Hand wash Wool 35 en Detergents and care products Symbol Washing process Recommended pro- gramme Not suitable for machine-washing – ¡ Do not mix different liquid deter- D1etergents and care products 2 Detergents and care gents products ¡ Do not mix detergent and fabric softener Detergents and care products ¡ Do not use products that have ex- Note: Please read the information in pired or are highly congealed the Safety → Page 4 and Preventing ¡ Do not use

In [ ]:
idx = 11
# This loop costs money, so be careful to spend the entire $0.06 price tag. No i will not share my api key with you.

for i in tqdm(range(0, 20), total =20, desc="Burning through the cash"):

    response = client.responses.create(
        model="gpt-4o",
        instructions= prompt,
        input=large_chunks_df.loc[idx,"CHUNK_TEXT"],
    )

    print("Chunk: \n", large_chunks_df.loc[idx,"CHUNK_TEXT"])
    print(response.output_text)
    print("\n"*2)

Burning through the cash:   5%|▌         | 1/20 [00:02<00:45,  2.41s/it]

Chunk: 
 he detergent drawer and 10.Clean the inside of the detergent the lid with a soft, damp cloth or drawer housing. hand shower. 11.Push in the detergent drawer. 9. Dry the detergent drawer, lid and pump unit and reinsert them. 18.4 Descaling ATTENTION! The use of unsuitable descaling agents, e.g. for coffee machines, may damage the appliance. ▶ For this appliance, only use de- scaling agents that are available from the manufacturer's website or our after-sales service. 18.5 Cleaning the drain pump Clean the drain pump in the event of faults, e.g. blockages or rattling noises. Emptying the drain pump Note: Please read the information in the Safety → Page 4 and Preventing material damage → Page 10 sections to ensure that you use the appliance safely. 1. Turn off the water tap. 2. Switch off the appliance. 3. Disconnect the appliance's mains plug from the power supply. 50 Cleaning and servicing en 4. Open the maintenance flap. 7. Take the drain hose out of the holder. 5. Remove the 

Burning through the cash:  10%|█         | 2/20 [00:04<00:40,  2.26s/it]

Chunk: 
 he detergent drawer and 10.Clean the inside of the detergent the lid with a soft, damp cloth or drawer housing. hand shower. 11.Push in the detergent drawer. 9. Dry the detergent drawer, lid and pump unit and reinsert them. 18.4 Descaling ATTENTION! The use of unsuitable descaling agents, e.g. for coffee machines, may damage the appliance. ▶ For this appliance, only use de- scaling agents that are available from the manufacturer's website or our after-sales service. 18.5 Cleaning the drain pump Clean the drain pump in the event of faults, e.g. blockages or rattling noises. Emptying the drain pump Note: Please read the information in the Safety → Page 4 and Preventing material damage → Page 10 sections to ensure that you use the appliance safely. 1. Turn off the water tap. 2. Switch off the appliance. 3. Disconnect the appliance's mains plug from the power supply. 50 Cleaning and servicing en 4. Open the maintenance flap. 7. Take the drain hose out of the holder. 5. Remove the 

Burning through the cash:  15%|█▌        | 3/20 [00:06<00:34,  2.01s/it]

...

Chunk: 
 he detergent drawer and 10.Clean the inside of the detergent the lid with a soft, damp cloth or drawer housing. hand shower. 11.Push in the detergent drawer. 9. Dry the detergent drawer, lid and pump unit and reinsert them. 18.4 Descaling ATTENTION! The use of unsuitable descaling agents, e.g. for coffee machines, may damage the appliance. ▶ For this appliance, only use de- scaling agents that are available from the manufacturer's website or our after-sales service. 18.5 Cleaning the drain pump Clean the drain pump in the event of faults, e.g. blockages or rattling noises. Emptying the drain pump Note: Please read the information in the Safety → Page 4 and Preventing material damage → Page 10 sections to ensure that you use the appliance safely. 1. Turn off the water tap. 2. Switch off the appliance. 3. Disconnect the appliance's mains plug from the power supply. 50 Cleaning and servicing en 4. Open the maintenance flap. 7. Take the drain hose out of the holder. 5. Remove the 

Burning through the cash:  95%|█████████▌| 19/20 [00:57<00:02,  2.84s/it]

Chunk: 
 he detergent drawer and 10.Clean the inside of the detergent the lid with a soft, damp cloth or drawer housing. hand shower. 11.Push in the detergent drawer. 9. Dry the detergent drawer, lid and pump unit and reinsert them. 18.4 Descaling ATTENTION! The use of unsuitable descaling agents, e.g. for coffee machines, may damage the appliance. ▶ For this appliance, only use de- scaling agents that are available from the manufacturer's website or our after-sales service. 18.5 Cleaning the drain pump Clean the drain pump in the event of faults, e.g. blockages or rattling noises. Emptying the drain pump Note: Please read the information in the Safety → Page 4 and Preventing material damage → Page 10 sections to ensure that you use the appliance safely. 1. Turn off the water tap. 2. Switch off the appliance. 3. Disconnect the appliance's mains plug from the power supply. 50 Cleaning and servicing en 4. Open the maintenance flap. 7. Take the drain hose out of the holder. 5. Remove the 

Burning through the cash: 100%|██████████| 20/20 [01:02<00:00,  3.12s/it]

Chunk: 
 he detergent drawer and 10.Clean the inside of the detergent the lid with a soft, damp cloth or drawer housing. hand shower. 11.Push in the detergent drawer. 9. Dry the detergent drawer, lid and pump unit and reinsert them. 18.4 Descaling ATTENTION! The use of unsuitable descaling agents, e.g. for coffee machines, may damage the appliance. ▶ For this appliance, only use de- scaling agents that are available from the manufacturer's website or our after-sales service. 18.5 Cleaning the drain pump Clean the drain pump in the event of faults, e.g. blockages or rattling noises. Emptying the drain pump Note: Please read the information in the Safety → Page 4 and Preventing material damage → Page 10 sections to ensure that you use the appliance safely. 1. Turn off the water tap. 2. Switch off the appliance. 3. Disconnect the appliance's mains plug from the power supply. 50 Cleaning and servicing en 4. Open the maintenance flap. 7. Take the drain hose out of the holder. 5. Remove the 